In [42]:
import requests
import re
from bs4 import BeautifulSoup as bsoup
import numpy as np
import datetime
from dateutil import parser
import math
import pandas as pd

In [49]:
STARTING_YEAR = 2016
now = datetime.datetime.now()
header = ['date', 'tick', 'export', 'import', 'system_load']
month = []
months = []
n_month = 1
for i in range(1, int((datetime.datetime.now().year - STARTING_YEAR)*12 + datetime.datetime.now().month)+1):
    month.append(math.ceil(i/12)+STARTING_YEAR-1)
    month.append(str(datetime.date(STARTING_YEAR, n_month , 1).month))
    n_month += 1
    if (n_month == 13):
        n_month = 1
    months.append(month)
    month = []

'''Kesobb modositani kell:'''
months = months[:-2]
'''Az utolso ket honap adatai meg nem elerhetok az interneten az altalam hasznalt modszerrel'''
print(months)
# print(int(datetime.datetime.now().strftime('%j')))
# months = []
# for i in range(now.month, 13):
#     months.append(i)
# for i in range(1, now.month):
#     months.append(i)
# print(months)


[[2016, '1'], [2016, '2'], [2016, '3'], [2016, '4'], [2016, '5'], [2016, '6'], [2016, '7'], [2016, '8'], [2016, '9'], [2016, '10'], [2016, '11'], [2016, '12'], [2017, '1'], [2017, '2']]


In [3]:
'''Az a blokk regebbi adatbazis keszitesehez kellett'''

#     for i in range(1, 4):
#         header = tables[i].find_all("tr")
#         # a header[0]-ban a tabla cime szerepel. Ez most nem erdekel minket
#         headings = [td.get_text() for td in header[1].find_all("td")]
#         '''@dataset: tarolja az orakra bontott adatokat'''
#         dataset = []
#         '''Az elso harom sor nem erdekes, mert: cim, header, es egy szeles vonal talalhato bennuk'''
#         for row in tables[i].find_all("tr")[3:]:
#             row_data=[]
#             for field in row.find_all("td"):
#                 row_data.append(field.get_text())                    
#             dataset.append(row_data)
#         dataset.insert(0, headings)
        
#         dataset = transform_table(headings, dataset)
#         #print(dataset)
        
#         #npdataset = np.array(dataset)
#         datasets.append(dataset)#npdataset)
#     database.append(datasets)


'''A parameterul kapott tablazatot az alabbiak szerint transzformalja.
    Egy sor adatai: [date, tick, import, export, system_load]
    A headings a html header datum ertekei
    '''
def transform_table(headings, dataset):
    transformed_row = []
    rowstruct = [0,0,0,0,0]
    dateformat = '%Y.%m.%d.'
    '''a 0. helyen "Idoszak (ora)" szerepel. Szamunkra ez nem erdekes'''
    for date in range(1, len(headings)):
        rowstruct[0] = datetime.datetime.strptime(headings[date], dateformat)#.timetuple().tm_yday
        for hour in range(1, len(dataset)-4):
            rowstruct[1] = dataset[hour][0].split('-')[0]
            rowstruct[2] = dataset[hour][date]
            transformed_row.append(rowstruct)
            rowstruct = [0,0,0]
            rowstruct[0] = datetime.datetime.strptime(headings[date], dateformat).timetuple().tm_yday
    return transformed_row 

In [50]:
''' @database: itt taroljuk majd a rendezett adatokat.
    12 honap kulon tarolodik benne, honaponkent az Export es Import tablazatok
    '''
database = []
for month in months:
    '''lekerjuk a kulonbozo honapokat'''
    URL = "https://www.mavir.hu/delegate/MonthlyView?lang=hu&year=" + str(month[0]) + "&month=" + str(month[1])
    print(URL)
    response = requests.get(URL)
    html = response.content.decode("utf-8").replace(u"\u0151", "o")
    
    soup = bsoup(html)
    soup.prettify()

    '''@datasets: tarolja az Export, Import, Rendszerterheles tablakat'''
    datasets = []
    tables = soup.find_all("table")
    '''Mivel minket csak az Export/Import/Rendszerterheles erdekel ezert,
        csak a 2., 3. es 4. tabla feldolgozasa a cel'''
    
    ''' a header[0]-ban a tabla cime szerepel. Ez most nem erdekel minket
        A html-ben minden tabla fejlece ugyanazokkal a datumokkal van cimkezve
        ezert eleg egy vizsgalata.
    '''
    
    date_header = [(td.get_text()+'.') for td in (tables[1].find_all('tr'))[1].find_all("td")][1:]
    print(date_header)
    ticks = [row.find_all('td')[0].get_text() for row in tables[1].find_all("tr", {"class" : "results-row"})]
    ticks = ticks[:-3]
    '''1: import, 2: export, 3: rendszerterheles'''
    import_ = tables[1].find_all("tr", {"class" : "results-row"})
    export_ = tables[2].find_all("tr", {"class" : "results-row"})
    system_load = tables[3].find_all("tr", {"class" : "results-row"})
    for date in range(0,len(date_header)):
        for tick in range(0,len(ticks)):
            row=[]
            row.append(date_header[date])
            row.append(ticks[tick])
            row.append(import_[tick].find_all('td')[date+1].get_text())
            row.append(export_[tick].find_all('td')[date+1].get_text())
            row.append(system_load[tick].find_all('td')[date+1].get_text())
            database.append(row)

database = pd.DataFrame(database, columns=header)
'''Kimentjuk a keletkezett adatbazist egy fajlba'''
print(database)
database.to_csv('database.csv', sep=',', index=False)

https://www.mavir.hu/delegate/MonthlyView?lang=hu&year=2016&month=1


/Users/krisz/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/krisz/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


['2016.01.01.', '2016.01.02.', '2016.01.03.', '2016.01.04.', '2016.01.05.', '2016.01.06.', '2016.01.07.', '2016.01.08.', '2016.01.09.', '2016.01.10.', '2016.01.11.', '2016.01.12.', '2016.01.13.', '2016.01.14.', '2016.01.15.', '2016.01.16.', '2016.01.17.', '2016.01.18.', '2016.01.19.', '2016.01.20.', '2016.01.21.', '2016.01.22.', '2016.01.23.', '2016.01.24.', '2016.01.25.', '2016.01.26.', '2016.01.27.', '2016.01.28.', '2016.01.29.', '2016.01.30.', '2016.01.31.']
https://www.mavir.hu/delegate/MonthlyView?lang=hu&year=2016&month=2
['2016.02.01.', '2016.02.02.', '2016.02.03.', '2016.02.04.', '2016.02.05.', '2016.02.06.', '2016.02.07.', '2016.02.08.', '2016.02.09.', '2016.02.10.', '2016.02.11.', '2016.02.12.', '2016.02.13.', '2016.02.14.', '2016.02.15.', '2016.02.16.', '2016.02.17.', '2016.02.18.', '2016.02.19.', '2016.02.20.', '2016.02.21.', '2016.02.22.', '2016.02.23.', '2016.02.24.', '2016.02.25.', '2016.02.26.', '2016.02.27.', '2016.02.28.', '2016.02.29.']
https://www.mavir.hu/delegate/